# Leaf Segmentation

### Importing required libraries:

In [9]:
import numpy as np
import cv2
import warnings
import os
import shutil
from matplotlib import pyplot as plt
import skimage
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops, regionprops_table
import pandas as pd

### Adding image path:

In [10]:
path = 'attempt1.jpg'#'./testimg.JPG'
presegpath='attempt1_segmented.jpg'
# path = './test2.JPG'
image = cv2.imread(path)
presegimg=cv2.imread(presegpath)
grayImage = cv2.cvtColor(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2BGR)
denoisedImage = cv2.fastNlMeansDenoisingColored(image)

### Function for removing everything except the leaf using masks on HSV:

In [11]:
def getSegmentationMask(image):
    #Creating mask for non-diseased part
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # cv2.imwrite('./Output/hsv.JPG',hsv)
    # lower_green = np.array ([30, 31, 22])
    # upper_green = np.array ([85, 235,  195])
    lower_green = (24,30,0)
    upper_green = (90,240,180)
    mask = cv2.inRange(hsv,lower_green,upper_green)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel=np.ones((8,8),dtype=np.uint8))
    # cv2.imwrite('./Output/greenMask.JPG',mask)
    #Creating mask for diseased parts
    low_val=(5,95, 0)
    high_val=(20, 255, 255)
    maskDisease = cv2.inRange(hsv,low_val,high_val)#lower,upper)#low_val,high_val)
    # cv2.imwrite('./Output/maskDiesease.JPG',maskDisease)

    # remove noise
    # maskDisease = cv2.morphologyEx(maskDisease, cv2.MORPH_CLOSE, kernel=np.ones((8,8),dtype=np.uint8))
    mask=mask+maskDisease
    # mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel=np.ones((8,8),dtype=np.uint8))

    #blur
    mask = cv2.medianBlur(mask,15)
    #Contour detection:
    conts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL ,cv2.CHAIN_APPROX_SIMPLE)
    conts = conts[0] #for openCV version 3.x and maybe below, it would be conts[1]
    c = sorted(conts , key = cv2.contourArea , reverse = True)[0]
    height , width = image.shape [:2]
    mask = cv2.drawContours(np.zeros((height ,width ,3), np.uint8), [c], 0, (255 ,255 ,255), cv2.FILLED)
    mask = cv2.cvtColor(mask ,cv2.COLOR_BGR2GRAY)
    # cv2.imwrite('finalMask.JPG',mask)
    return [mask,maskDisease]


### Function for getting segmented image:

In [12]:
def getSegmentedImage(image):
    [mask,maskDisease] = getSegmentationMask(image)
    segmentedImg = cv2.bitwise_and(image,image,mask=mask)
    # # segmentedImg = cv2.bitwise_and(segmentedImg,cv2.bitwise_not(mask2))
    # # segmentedImg = cv2.bitwise_and(segmentedImg,cv2.bitwise_not(mask3))
    # cv2.imshow("Mask: ",mask)
    # cv2.imshow("Disease Mask: ",maskDisease)
    # cv2.imshow("Masked: ",segmentedImg)
    # cv2.imshow('Original: ',image)
    # # cv2.imshow("Grayscale: ",grayImage)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return segmentedImg

### Comparing pre segmented and our segmented:

In [13]:
import re 
def findbackground(path):
    return re.sub(r'color_images','ground_truth',path)

def Dice(segmentation,ground):
    
    
    intersection=cv2.bitwise_and(segmentation,ground)
    intersection=np.count_nonzero(intersection>0)
    A=np.count_nonzero(segmentation>0)
    B=np.count_nonzero(ground>0)
    return 2 * intersection /(A+B)

def IntersectionOverUnion(segmentation,ground):
    intersection=cv2.bitwise_and(segmentation,ground)
    intersection=np.count_nonzero(intersection>0)
    image_union=cv2.bitwise_or(segmentation,ground)
    image_union=np.count_nonzero(image_union>0)
    return intersection/image_union

def measureAccuracy(segmentedImg,preSegmentedImg):
    IoU=IntersectionOverUnion(segmentedImg,preSegmentedImg)
    DiceCoeff = Dice(segmentedImg,preSegmentedImg)
    pd
    # print('IoU', IoU)
    # print('Dice', DiceCoeff)
    return [IoU,DiceCoeff]


### Execute the main part:

In [14]:
sumIoU=0
sumDice=0
imgCount=0
unsegBaseDir = './Dataset - Unsegmented/'
segBaseDir = './Dataset - Segmented/'
outputDir = os.path.abspath('./Output')
if(os.path.exists(outputDir)):
    shutil.rmtree(outputDir)
os.mkdir(outputDir)
df = pd.DataFrame(columns=['Original Image','IoU','Dice'])
with warnings.catch_warnings():
    warnings.simplefilter("ignore",category=FutureWarning)
    for filename in os.scandir(unsegBaseDir):
        if filename.is_file():
            # print(filename.name)
            currSegImgDir=segBaseDir+str.replace(filename.name,'.JPG','_final_masked.JPG')
            currImg = cv2.imread(filename.path)
            currSegImg = cv2.imread(currSegImgDir)
            ourSegImg = getSegmentedImage(currImg)
            [IoU,DiceC] = measureAccuracy(ourSegImg,currSegImg)
            df = df.append({'Original Image':filename.name,'IoU':IoU,'Dice':DiceC},ignore_index=True)
            sumIoU+=IoU
            sumDice+=DiceC
            imgCount+=1
            cv2.imwrite(outputDir+'/'+str.replace(filename.name,'.JPG','_MY_SEGMENTED.JPG'), ourSegImg)
# print(lastImgDir)
# img=cv2.imread(lastImgDir)
# cv2.imshow('Img:',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
AvgDice = sumDice/imgCount
AvgIoU=sumIoU/imgCount
print(f"{AvgIoU = }\n{AvgDice = }\n{imgCount = }")
print({'Average Dice':AvgDice,'Average IoU':AvgIoU,'Number of Images':imgCount})

AvgIoU = 0.781666479688154
AvgDice = 0.8403977039003053
imgCount = 1383
{'Average Dice': 0.8403977039003053, 'Average IoU': 0.781666479688154, 'Number of Images': 1383}


In [15]:
print(df.sort_values('IoU', ascending=False))
print(df['IoU'].mean())
print(df['IoU'].median())

                                         Original Image       IoU      Dice
588   6de1674f-2bea-4388-9a5c-d40ff2fa44fc___FAM_B.M...  0.870903  0.903124
1137  d2d21ea0-7616-4f7a-9b35-1909d72de459___FAM_B.M...  0.856063    0.8963
536   63f3535c-0927-442e-a4ee-ffce62296cb9___FAM_B.M...   0.85108   0.88813
874   a4b05803-14e4-4551-acb5-ace444f7dac4___FAM_B.M...  0.849156  0.892378
908   ab8a8cb6-8fd5-4bb4-934a-a37dabacd13b___FAM_B.M...  0.848467   0.89153
...                                                 ...       ...       ...
797   95308c47-b376-46b5-b184-61ace51c2245___FAM_B.M...  0.651918  0.759856
8     02223cc6-30d2-408d-a091-65257d044a50___FAM_B.M...  0.651031  0.750796
669   7b47e162-0e3e-41c7-93fe-63a1466d7609___FAM_B.M...   0.64327  0.751771
175   20ab9746-0822-4b1b-9785-759d35ffff95___FAM_B.M...  0.537574  0.675921
774   8f9970b4-19ad-45cd-8544-1a01360c62cf___FAM_B.M...   0.52861  0.663431

[1383 rows x 3 columns]
0.781666479688154
0.785231212230284


In [16]:
print(df.sort_values('Dice', ascending=False))
print(df['Dice'].mean())
print(df['Dice'].median())

                                         Original Image       IoU      Dice
588   6de1674f-2bea-4388-9a5c-d40ff2fa44fc___FAM_B.M...  0.870903  0.903124
1137  d2d21ea0-7616-4f7a-9b35-1909d72de459___FAM_B.M...  0.856063    0.8963
874   a4b05803-14e4-4551-acb5-ace444f7dac4___FAM_B.M...  0.849156  0.892378
908   ab8a8cb6-8fd5-4bb4-934a-a37dabacd13b___FAM_B.M...  0.848467   0.89153
315   3ac35d46-56c4-4137-b7a3-7023dc0beb41___FAM_B.M...  0.846008  0.889725
...                                                 ...       ...       ...
975   b90c684d-49c7-4e9f-88a2-8b8b726f4b90___FAM_B.M...  0.653592  0.756452
669   7b47e162-0e3e-41c7-93fe-63a1466d7609___FAM_B.M...   0.64327  0.751771
8     02223cc6-30d2-408d-a091-65257d044a50___FAM_B.M...  0.651031  0.750796
175   20ab9746-0822-4b1b-9785-759d35ffff95___FAM_B.M...  0.537574  0.675921
774   8f9970b4-19ad-45cd-8544-1a01360c62cf___FAM_B.M...   0.52861  0.663431

[1383 rows x 3 columns]
0.8403977039003053
0.84257407568424
